In [1]:
!pip install -q torch torchvision ultralytics nltk pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
MODEL_PATH = "/content/drive/MyDrive/FYP_Models/Image_Captioning/resnet50_model.pth"
VOCAB_PATH = "/content/drive/MyDrive/FYP_Models/Image_Captioning/vocab.pt"

In [5]:
word2idx, idx2word = torch.load(VOCAB_PATH)
vocab_size = len(word2idx)
print("✅ Vocabulary loaded:", vocab_size)

✅ Vocabulary loaded: 3004


In [6]:
class CaptionModel(nn.Module):
    def __init__(self, vocab_size, feature_size=2048, embed_size=256, hidden_size=512):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.feature_fc = nn.Linear(feature_size, hidden_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions):
        h0 = self.feature_fc(features).unsqueeze(0).unsqueeze(0)
        c0 = torch.zeros_like(h0)
        emb = self.embedding(captions)
        out, _ = self.lstm(emb, (h0, c0))
        return self.fc(out)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CaptionModel(vocab_size).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

print("✅ Caption model loaded from Drive")

✅ Caption model loaded from Drive


In [8]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval().to(device)

for p in resnet.parameters():
    p.requires_grad = False

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 159MB/s]


In [9]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [10]:
OBJ_EMBED = nn.Embedding(1000, 256).to(device)
fusion_fc = nn.Linear(2048 + 256, 2048).to(device)

def fuse_features(image_path):
    image = Image.open(image_path).convert("RGB")
    img_t = transform(image).unsqueeze(0).to(device)

    # ResNet features
    with torch.no_grad():
        visual_feat = resnet(img_t).squeeze()

    return visual_feat # Return only visual_feat

In [11]:
import torch.nn.functional as F

def generate_caption(feature, max_len=20, temperature=0.8):
    model.eval()

    with torch.no_grad():
        h = model.feature_fc(feature).unsqueeze(0).unsqueeze(0)
        c = torch.zeros_like(h)

        # Use the SAME tokenization as training
        start_tokens = [
            word2idx['<'],
            word2idx['start'],
            word2idx['>']
        ]

        word = torch.tensor([start_tokens]).to(device)
        result = []

        for step in range(max_len):
            emb = model.embedding(word[:, -1:])
            out, (h, c) = model.lstm(emb, (h, c))
            scores = model.fc(out.squeeze(1))

            probs = F.softmax(scores / temperature, dim=1)
            pred = torch.multinomial(probs, 1).item()
            predicted_word = idx2word[pred]

            # stop condition
            if predicted_word == 'end':
                break

            # avoid printing special tokens
            if predicted_word not in {'<', 'start', '>'}:
                result.append(predicted_word)

            word = torch.cat(
                [word, torch.tensor([[pred]]).to(device)],
                dim=1
            )

        return " ".join(result)

In [12]:
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

Saving dog.png to dog.png


**BEAM SEARCH**

In [14]:
import torch.nn.functional as F

def generate_caption_beam(feature, beam_size=3, max_len=20):
    model.eval()

    with torch.no_grad():
        h = model.feature_fc(feature).unsqueeze(0).unsqueeze(0)
        c = torch.zeros_like(h)

        # (sequence, score, h, c)
        # The sequence here will be a list of token IDs
        start_tokens = [word2idx['<'], word2idx['start'], word2idx['>']]
        beams = [(start_tokens, 0.0, h, c)] # (sequence_of_ids, log_prob_score, hidden_state, cell_state)

        # To store completed captions
        final_captions = []

        for _ in range(max_len):
            if not beams: # All beams have completed
                break

            new_beams = []
            for seq, score, h_state, c_state in beams:
                # If this beam already ended (from previous step), just add it to final_captions and don't extend
                if idx2word[seq[-1]] == 'end':
                    final_captions.append((seq, score))
                    continue

                last_word_tensor = torch.tensor([[seq[-1]]]).to(device)
                emb = model.embedding(last_word_tensor)
                out, (h_next, c_next) = model.lstm(emb, (h_state, c_state))
                logits = model.fc(out.squeeze(1))
                probs = F.log_softmax(logits, dim=1)

                topk_probs, topk_idx = probs.topk(beam_size)

                for i in range(beam_size):
                    next_word_idx = topk_idx[0][i].item()
                    new_score = score + topk_probs[0][i].item()
                    new_seq = seq + [next_word_idx]

                    if idx2word[next_word_idx] == 'end':
                        final_captions.append((new_seq, new_score))
                    else:
                        new_beams.append((new_seq, new_score, h_next, c_next))

            # Select the top beam_size beams for the next iteration (prioritizing higher scores)
            # This step keeps only active beams and sorts them for the next iteration
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]

        # After the loop, if there are any remaining active beams, treat them as complete
        final_captions.extend([(seq, score) for seq, score, _, _ in beams])

        if not final_captions:
            # Fallback if no captions were completed (this shouldn't happen with reasonable max_len)
            # If this occurs, it indicates an issue, so we'll return a placeholder string.
            return "Generation failed to complete."

        # Sort all completed captions (including those that hit max_len) and pick the best one
        best_seq_with_score = sorted(final_captions, key=lambda x: x[1], reverse=True)[0]
        best_seq = best_seq_with_score[0]

        caption_words = []
        for idx in best_seq:
            word = idx2word[idx]
            if word in {'<', 'start', '>', 'end'}:
                continue
            caption_words.append(word)

        return " ".join(caption_words)

In [15]:
feature = fuse_features(image_path)
print("🖼️ Caption (Beam):", generate_caption_beam(feature))

🖼️ Caption (Beam): a brown and white dog is running on the grass .
